In [2]:
import numpy as np
from sklearn.pipeline import Pipeline
from python.utils import load_data, extract_and_save, get_features_from 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from python.features_extraction import ExtractFeatures
from sklearn.model_selection import cross_validate

In [3]:
fc_parameters = {
    "abs_energy": None,
    "maximum": None,
    "absolute_sum_of_changes": None,
    "fft_coefficient": [{"coeff": 10, "attr": "abs"}],
    "fourier_entropy": [{"bins": 10}],
    "linear_trend": [{"attr": "slope"}],
    "mean": None,
    "number_peaks": [{"n":4}],
    "variance": None,
}

In [3]:
DATA_PATH = 'data'
X_train, y_train, X_test, subjects = load_data(DATA_PATH)

X_train size: (3500, 15872).
y_train size: (3500,).
X_test size: (3500, 15872).


In [ ]:
pre_process = Pipeline([
    ("si", SimpleImputer(missing_values=-999999.99, strategy="mean")),
    ("pp", ExtractFeatures(fc_parameters)),
    ("si2", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("ss", StandardScaler())
])

In [9]:
model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10), n_estimators=200)
feature_extract_model = Pipeline([
    ("pp", pre_process),
    ("ada_cls", model)
])
cv_res = cross_validate(feature_extract_model, X_train, y_train, groups=subjects, cv=GroupKFold(len(np.unique(subjects))), return_train_score=True)
cv_res['test_score'].mean()